# SQLite to CSV
Script that allows you to export individual tables into csv files.

In [2]:
import sqlite3
import pandas as pd
import os

# Path
db_path = '../../PJM-sign-language.db'

output_dir = output_dir = os.getcwd() + '/export'

# Check if exists
os.makedirs(output_dir, exist_ok=True)

# Connect to db
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Select tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Export for each table
for table_name_tuple in tables:
    table_name = table_name_tuple[0]
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    output_file = os.path.join(output_dir, f"{table_name}.csv")
    df.to_csv(output_file, index=False)
    print(f"Zapisano: {output_file}")

# Close conn
conn.close()


Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/Z.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/Z_Z.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/Z_X.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/A.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/B.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/C.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/E.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/A_A.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/C_C.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/D.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/E_E.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/F.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dataset/export/G.csv
Zapisano: /home/magda/Documents/Github/project-pjm/dat

In [4]:
import sqlite3
import csv

def export_all_tables_to_single_csv(db_file: str, output_csv: str) -> None:
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]

    global_sample_id = 0
    merged_rows = []
    header = None

    for table in tables:
        cursor.execute(f"SELECT * FROM {table} ORDER BY sample_id, frame;")
        rows = cursor.fetchall()

        if header is None:
            header = [desc[0] for desc in cursor.description]

        current_sample = None
        for row in rows:
            row = list(row)
            local_sample_id = row[header.index('sample_id')]

            if local_sample_id != current_sample:
                current_sample = local_sample_id
                global_sample_id += 1

            row[header.index('sample_id')] = global_sample_id
            merged_rows.append(row)

    with open(output_csv, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(merged_rows)

    conn.close()
    print(f"All tables merged and exported to {output_csv}")

# Użycie:
export_all_tables_to_single_csv("../../PJM-sign-language.db", "own_dataset.csv")


All tables merged and exported to own_dataset.csv
